In [2]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU

# Set the path to your dataset directory
dataset_path = './dataset/train'

# Define the image size for resizing
image_size = (32, 32)

# Load and preprocess the dataset
X_data = []
y_data = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_data.append(image)
                y_data.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_data = np.array(X_data)
y_data = np.array(y_data)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# Encode the shape labels into numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the numerical labels to one-hot encoded vectors
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes)

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

# Define a new data point to classify
new_data = np.array([cv2.resize(cv2.imread('./dataset/new/newshape.jpg'), image_size)])

# Predict the label for the new data point
predicted_probabilities = model.predict(new_data)
predicted_label = label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1))

# Print the predicted label
print("Predicted shape:", predicted_label)


Epoch 1/10
188/188 [==============================] - 8s 33ms/step - loss: 5.5088 - accuracy: 0.2360 - val_loss: 1.5020 - val_accuracy: 0.3320
Epoch 2/10
188/188 [==============================] - 6s 31ms/step - loss: 1.1628 - accuracy: 0.5370 - val_loss: 0.6821 - val_accuracy: 0.7320
Epoch 3/10
188/188 [==============================] - 5s 29ms/step - loss: 0.4161 - accuracy: 0.8450 - val_loss: 0.3269 - val_accuracy: 0.8900
Epoch 4/10
188/188 [==============================] - 6s 34ms/step - loss: 0.1955 - accuracy: 0.9288 - val_loss: 0.1992 - val_accuracy: 0.9287
Epoch 5/10
188/188 [==============================] - 6s 30ms/step - loss: 0.1255 - accuracy: 0.9532 - val_loss: 0.1843 - val_accuracy: 0.9373
Epoch 6/10
188/188 [==============================] - 6s 30ms/step - loss: 0.1021 - accuracy: 0.9622 - val_loss: 0.1884 - val_accuracy: 0.9333
Epoch 7/10
188/188 [==============================] - 6s 31ms/step - loss: 0.0750 - accuracy: 0.9717 - val_loss: 0.2711 - val_accuracy: 0.9160

# Check Predicted Shape

In [4]:
# Define a new data point to classify
new_data = np.array([cv2.resize(cv2.imread('./dataset/new/newshape.jpg'), image_size)])

# Predict the label for the new data point
predicted_probabilities = model.predict(new_data)
predicted_label = label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1))

# Print the predicted label
print("Predicted shape:", predicted_label)

1/1 [==============================] - 0s 43ms/step
Predicted shape: ['trapezoid']


In [7]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

47/47 [==============================] - 0s 9ms/step - loss: 0.1319 - accuracy: 0.9533
Accuracy: 0.95333331823349
